# Import des libraries

In [ ]:
!git clone https://github.com/fundamentalvision/Deformable-DETR.git
!cd /content/Deformable-DETR
!pip install -r requirements.txt


fatal: destination path 'Deformable-DETR' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
%cd Deformable-DETR
!pip install -r requirements.txt

/content/Deformable-DETR


# Importation

In [ ]:
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as T
from pycocotools.coco import COCO
from torchvision.datasets import CocoDetection
from transformers import AutoImageProcessor, DeformableDetrForObjectDetection
import numpy as np
import random

# Préparation du dataset

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data_raw

```
data/
├── images/
│   ├── train/
│   │   ├── 000000000001.jpg
│   │   ├── 000000000002.jpg
│   │   ├── ...
│   └── val/
│       ├── 000000000001.jpg
│       ├── 000000000002.jpg
│       ├── ...
├── annotations/
│   ├── instances_train2017.json
│   ├── instances_val2017.json


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Répertoire d'origine pour les images et annotations
raw_data_path_images = "/content/custom_data_raw/images"  # Images sont dans ce dossier
raw_data_path_labels = "/content/custom_data_raw/obj_train_data"  # Annotations dans ce dossier

# Obtenir toutes les images (jpg/png) présentes dans le dossier "images"
images = [f for f in os.listdir(raw_data_path_images) if f.endswith(('.jpg', '.png'))]

# Split train/val (80% train, 20% val)
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# Création des dossiers cible pour images et labels
base_path = "/content/custom_data"


os.makedirs(base_path + "/images/train", exist_ok=True)
os.makedirs(base_path + "/images/val", exist_ok=True)
os.makedirs(base_path + "/labels/train", exist_ok=True)
os.makedirs(base_path + "/labels/val", exist_ok=True)

def move_data(image_list, split):
    for img_name in image_list:
        # Déplacer l'image vers le bon dossier
        src_img = os.path.join(raw_data_path_images, img_name)
        dst_img = os.path.join(base_path, f"images/{split}", img_name)
        shutil.copy(src_img, dst_img)

        # Vérifier et déplacer l'annotation .txt correspondante
        txt_name = img_name.rsplit('.', 1)[0] + '.txt'
        txt_src = os.path.join(raw_data_path_labels, txt_name)
        txt_dst = os.path.join(base_path, f"labels/{split}", txt_name)

        if os.path.exists(txt_src):
            shutil.copy(txt_src, txt_dst)
        else:
            print(f"Pas d'annotation pour {img_name}")

# Appliquer le déplacement aux deux splits : train et val
move_data(train_imgs, "train")
move_data(val_imgs, "val")


In [ ]:
import os
import cv2
import json
from tqdm import tqdm

tile_size = 320
overlap = 0
splits = ["train", "val"]

input_root = "/content/custom_data"
output_root = "/content/split_custom_data"

categories = []  # remplie dynamiquement

def yolo_to_bbox(x_center, y_center, w, h, img_w, img_h):
    x1 = (x_center - w / 2) * img_w
    y1 = (y_center - h / 2) * img_h
    w *= img_w
    h *= img_h
    return x1, y1, w, h  # Format COCO: x, y, width, height

for split in splits:
    image_id = 0
    annotation_id = 0

    input_img_dir = os.path.join(input_root, f"images/{split}")
    input_lbl_dir = os.path.join(input_root, f"labels/{split}")
    output_img_dir = os.path.join(output_root, f"{split}/images")
    output_ann_path = os.path.join(output_root, f"{split}/instances_{split}.json")

    os.makedirs(output_img_dir, exist_ok=True)

    coco_dict = {
        "images": [],
        "annotations": [],
        "categories": [],
    }

    label_set = set()
    for filename in tqdm(os.listdir(input_img_dir), desc=f"{split}"):
        if not filename.lower().endswith(('.jpg', '.png')):
            continue

        basename = os.path.splitext(filename)[0]
        img_path = os.path.join(input_img_dir, filename)
        label_path = os.path.join(input_lbl_dir, f"{basename}.txt")

        img = cv2.imread(img_path)
        if img is None:
            continue
        img_h, img_w = img.shape[:2]

        annots = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, xc, yc, w, h = map(float, parts)
                        annots.append((int(cls), *yolo_to_bbox(xc, yc, w, h, img_w, img_h)))
                        label_set.add(int(cls))

        step = tile_size - overlap
        tile_id = 0

        for y in range(0, img_h, step):
            for x in range(0, img_w, step):
                tile = img[y:y+tile_size, x:x+tile_size]
                th, tw = tile.shape[:2]
                if th < tile_size or tw < tile_size:
                    continue

                tile_fname = f"{basename}_{tile_id}.jpg"
                tile_path = os.path.join(output_img_dir, tile_fname)
                cv2.imwrite(tile_path, tile)

                coco_dict["images"].append({
                    "id": image_id,
                    "width": tile_size,
                    "height": tile_size,
                    "file_name": tile_fname,
                })

                for cls, x1, y1, w, h in annots:
                    x2 = x1 + w
                    y2 = y1 + h

                    # Check if box overlaps the tile
                    if x1 >= x + tile_size or x2 <= x or y1 >= y + tile_size or y2 <= y:
                        continue

                    box_x1 = max(0, x1 - x)
                    box_y1 = max(0, y1 - y)
                    box_x2 = min(tile_size, x2 - x)
                    box_y2 = min(tile_size, y2 - y)

                    box_w = box_x2 - box_x1
                    box_h = box_y2 - box_y1

                    if box_w < 1 or box_h < 1:
                        continue

                    coco_dict["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": cls,
                        "bbox": [box_x1, box_y1, box_w, box_h],
                        "area": box_w * box_h,
                        "iscrowd": 0,
                    })
                    annotation_id += 1

                image_id += 1
                tile_id += 1

    coco_dict["categories"] = [
        {"id": cls, "name": str(cls), "supercategory": "none"}
        for cls in sorted(label_set)
    ]

    with open(output_ann_path, "w") as f:
        json.dump(coco_dict, f, indent=2)

    print(f"✅ COCO json sauvegardé dans : {output_ann_path}")


train: 100%|██████████| 28/28 [00:04<00:00,  6.31it/s]


✅ COCO json sauvegardé dans : /content/split_custom_data/train/instances_train.json


val: 100%|██████████| 7/7 [00:01<00:00,  6.33it/s]

✅ COCO json sauvegardé dans : /content/split_custom_data/val/instances_val.json


In [ ]:
!ls /content/split_custom_data/train/images | head
!ls /content/split_custom_data/train | grep json


20221108_112523_0.jpg
20221108_112523_10.jpg
20221108_112523_11.jpg
20221108_112523_12.jpg
20221108_112523_13.jpg
20221108_112523_14.jpg
20221108_112523_15.jpg
20221108_112523_16.jpg
20221108_112523_17.jpg
20221108_112523_18.jpg
instances_train.json


Les étapes précèdentes sont là afin que le dataset possède le format voulu.

# Training


In [ ]:
dataset_path = "/content/split_custom_data"
output_path = "/content/output_olives"

In [ ]:
!ls

benchmark.py  datasets	engine.py  LICENSE  models     requirements.txt  util
configs       docs	figs	   main.py  README.md  tools


In [ ]:
!pip install torch==1.12.1 torchvision==0.13.1

ERROR: Could not find a version that satisfies the requirement torch==1.12.1 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.12.1


In [ ]:
!python main.py \
  --dataset_file coco \
  --coco_path {dataset_path} \
  --output_dir {output_path} \
  --batch_size 2 \
  --epochs 50 \
  --lr_drop 40 \
  --num_workers 2 \
  --resume "" \
  --with_box_refine \
  --two_stage


Traceback (most recent call last):
  File "/content/Deformable-DETR/main.py", line 21, in <module>
    import datasets
  File "/content/Deformable-DETR/datasets/__init__.py", line 13, in <module>
    from .coco import build as build_coco
  File "/content/Deformable-DETR/datasets/coco.py", line 22, in <module>
    from util.misc import get_local_rank, get_local_size
  File "/content/Deformable-DETR/util/misc.py", line 32, in <module>
    from torchvision.ops.misc import _NewEmptyTensorOp
ImportError: cannot import name '_NewEmptyTensorOp' from 'torchvision.ops.misc' (/usr/local/lib/python3.11/dist-packages/torchvision/ops/misc.py)


# Sauvegarde

In [ ]:
torch.save(model.state_dict(), 'deformable_detr.pth')